
---
## Projectr : 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보분석 ver.01
### Description : 
    - 스마트서울 도시데이터 센서(S-DoT) 유동인구 측정 정보
    [source link](https://data.seoul.go.kr/dataList/OA-15964/S/1/datasetView.do)
    - 센서수량 및 설치장소 : 서울 전역 100대(전통시장, 주요거리, 공원, 공공시설)
    - 측정정보 : S-DoT에서 측정된 유동인구(10분단위 데이터)
    - 유동인구 센서는 WiFi AP셀 반경내 휴대폰의 MAC주소를 카운팅(50대)하거나 CCTV를 활용한 피플카운팅(50대) 방식으로 정확한 유동인구를 측정한 것이 아닙니다.
    - 데이터의 양이 많아 2023년 10월 16일부터 Sheet와 OpenAPI는 최근 한달만 출력됩니다.
### Author : Forrest Dpark (분석 담당)
### Date : 2024.05.31 ~
### Detail :  
### Update: 
    > ※ 2024.06.03 pdg : 데이터 체크 
        - api 연결
---

In [1]:
import pandas as pd ,numpy as  np # Data manipulatioin
import seaborn as sns,matplotlib.pyplot as plt # visiulization
import warnings;warnings.filterwarnings('ignore') # 경고 무시 
# 기본 세팅
def plotSetting(pltStyle):
    import matplotlib.pyplot as plt # visiulization
    import platform
    from matplotlib import font_manager, rc # rc : 폰트 변경 모듈font_manager : 폰트 관리 모듈
    plt.style.use(pltStyle)
    plt.rcParams['axes.unicode_minus'] = False# unicode 설정
    if platform.system() == 'Darwin': rc('font', family='AppleGothic') # os가 macos
    elif platform.system() == 'Windows': # os가 windows
        path = 'c:/Windows/Fonts/malgun.ttf' 
        font_name = font_manager.FontProperties(fname=path).get_name()
        rc('font', family=font_name)
    else:
        print("Unknown System")
    print("___## OS platform 한글 세팅완료 ## ___")
# graph style seaborn
plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___
